In [1]:
import dataset

ds = dataset.Dataset(batch_size=512)

Using TensorFlow backend.


In [4]:
from keras.models import Model
from keras.layers import (LSTM, Input, Embedding, concatenate, BatchNormalization, 
                          Activation, Conv2D, Reshape, Flatten, Dense)
from IPython.core.debugger import set_trace
# so I have 32, 32, 3 and I want 32, 32
# I can either add the three channels and divide (info is lost)
# or I can get it down to one channel from conv

def create_LSTM_net(magnitude = 1, lstm_layers=3, conv_layers=3, lstm_cells=3, conv_cells = 3, lstm_cell_size=3,
                    conv_kernel_size=3, conv_filter_size=3, input_shape=(32, 32, 3)):
    lstm_layers *= magnitude
    conv_layers *= magnitude
    lstm_cells *= magnitude
    conv_cells *= magnitude
#     conv_kernel_size *= magnitude
    conv_filter_size *= magnitude
    
    def CONV_CELL(prev, conv_filter_size):
        return Conv2D(conv_filter_size, conv_kernel_size, activation="relu")(prev)
    
    def CONV_LAYERS(prev, num_layers, num_cells):
        prev = BatchNormalization()(prev)
        
        for i in range(num_layers):
            cells = []
            for j in range(num_cells):
                cells.append(CONV_CELL(prev, conv_filter_size))
                
            concat = concatenate(cells, axis = -1)
            bn = BatchNormalization()(concat)
            activation = Activation("relu")(bn)
            prev = activation

        activation = Conv2D(1, 1)(activation)
        sub = int((num_layers*2*((conv_kernel_size-1)/2)))
        activation = Reshape((input_shape[0]-sub, input_shape[0]-sub))(activation)
            
        return activation
    
    def LSTM_CELL(prev):
        return LSTM(lstm_cell_size, return_sequences = True)(prev)

    def LSTM_LAYERS(prev, num_layers, num_cells):
        for i in range(num_layers):
            cells = []
            for j in range(num_cells):
                cells.append(LSTM_CELL(prev))
                
            concat = concatenate(cells, axis = -1)
            bn = BatchNormalization()(concat)
            activation = Activation("relu")(bn)
            prev = activation

        return bn

    inp = Input(shape=input_shape)
    
    layers = CONV_LAYERS(inp, conv_layers, conv_cells)
    layers = LSTM_LAYERS(layers, lstm_layers, lstm_cells)

    out = Flatten()(layers)
    out = Dense(10, activation = "softmax")(out)

    model = Model(inputs=inp, outputs=out)
    
    return model

In [5]:
from keras.optimizers import SGD, Nadam
from clr_callback import CyclicLR

def fit(model):
    optim = Nadam()
    base_lr = 0.001
    max_lr = 0.006
    clr = CyclicLR(base_lr=base_lr, max_lr=max_lr,
                   step_size=2000., mode='triangular')

    model.compile(optimizer=optim,
                  loss="categorical_crossentropy", metrics=["acc"])

    model.fit_generator(ds.train_gen, ds.train_steps, validation_data=ds.val_gen, validation_steps=ds.val_steps,
                        epochs=50, callbacks=[clr], verbose=1)

In [8]:
net = create_LSTM_net(conv_cells=9, conv_filter_size=9, lstm_cell_size=5, 
                      lstm_layers=5, conv_layers=3)

In [10]:
net.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
batch_normalization_14 (BatchNor (None, 32, 32, 3)     12          input_2[0][0]                    
____________________________________________________________________________________________________
conv2d_38 (Conv2D)               (None, 30, 30, 9)     252         batch_normalization_14[0][0]     
____________________________________________________________________________________________________
conv2d_39 (Conv2D)               (None, 30, 30, 9)     252         batch_normalization_14[0][0]     
___________________________________________________________________________________________

In [ ]:
fit(net)

Epoch 1/50
33/97 [=========>....................] - ETA: 1151s - loss: 2.4290 - acc: 0.1633